# Combine analysis results

In [ ]:
import pandas as pd
import respiration.utils as utils

analysis_dir = utils.dir_path('outputs', 'analysis')

## Harmonize the data

All respiration extraction methods have slightly different data structures. We need to harmonize the data to be able to compare the models. Only the best performing method for each model is kept.

In [ ]:
unsupervised_path = utils.join_paths(analysis_dir, 'unsupervised_analysis.csv')

unsupervised = pd.read_csv(unsupervised_path)

# Only keep roi==chest, because it is the most accurate
unsupervised = unsupervised[unsupervised['roi'] == 'chest']

# Remove roi column
unsupervised = unsupervised.drop(columns=['roi'])

# Rename method to model
unsupervised = unsupervised.rename(columns={'method': 'model'})

In [ ]:
fine_tuned_path = utils.join_paths(analysis_dir, 'fine_tuned_analysis.csv')

fine_tuned = pd.read_csv(fine_tuned_path)
# Rename model_id to model
fine_tuned = fine_tuned.rename(columns={'model_id': 'model'})

In [ ]:
raft_path = utils.join_paths(analysis_dir, 'raft_analysis.csv')
raft = pd.read_csv(raft_path)

# Only keep roi==chest, because it is the most accurate
raft = raft[raft['roi'] == 'chest']
raft = raft[raft['signal_direction'] == 'signal_v']

# Remove roi column
raft = raft.drop(columns=['roi'])

In [ ]:
pretrained_path = utils.join_paths(analysis_dir, 'pretrained_analysis.csv')
pretrained = pd.read_csv(pretrained_path)

In [ ]:
# Merge all dataframes
analysis = pd.concat([fine_tuned, pretrained, unsupervised, raft])

## Compare all models

In [ ]:
analysis['error'] = (analysis['prediction'] - analysis['ground_truth']).abs()

In [ ]:
analysis

In [ ]:
models = analysis['model'].unique()
metrics = analysis['metric'].unique()

In [ ]:
import scipy.stats as stats

correlations = []

for model in models:
    model_data = analysis[analysis['model'] == model]

    for metric in metrics:
        metric_data = model_data[model_data['metric'] == metric]

        if len(metric_data) != 0:
            correlation, p_value = stats.pearsonr(metric_data['prediction'], metric_data['ground_truth'])

            correlations.append({
                'model': model,
                'metric': metric,
                'correlation': correlation,
                'p_value': p_value,
            })

correlations = pd.DataFrame(correlations)
correlations

In [ ]:
# Plot the Ground Truth vs Prediction for each model
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(models[:3]), 1, figsize=(20, 20))

# Add some space between the plots
fig.tight_layout(pad=10.0)

fig.suptitle('Bland-Altman plots')

for idx, model in enumerate(models[:3]):
    ax = axes[idx]
    model_data = analysis[analysis['model'] == model]

    for metric in metrics:
        metric_data = model_data[model_data['metric'] == metric]

        if len(metric_data) != 0:
            ax.scatter(metric_data['prediction'], metric_data['error'], label=metric)

    ax.set_title(model)
    ax.set_xlabel('Prediction')
    ax.set_ylabel('Error')

    ax.legend()

In [ ]:
model_scores = []

for model in analysis['model'].unique():
    model_data = analysis[analysis['model'] == model]

    for metric in metrics:
        metric_data = model_data[model_data['metric'] == metric]
        if len(metric_data) != 0:
            model_scores.append({
                'model': model,
                'metric': metric,
                'mean': metric_data['error'].mean(),
                'std': metric_data['error'].std(),
            })

model_scores = pd.DataFrame(model_scores)
model_scores

In [ ]:
# Store the model scores
output_path = utils.dir_path('outputs', 'analysis', mkdir=True)
model_scores.to_csv(utils.join_paths(output_path, 'model_scores.csv'), index=False)

In [ ]:
# Determine the best model for each metric
best_models = []

for metric in metrics:
    metric_data = model_scores[model_scores['metric'] == metric]

    if metric == "distance_pearson":
        value = metric_data['mean'].max()
    else:
        value = metric_data['mean'].min()

    best_model = metric_data[metric_data['mean'] == value]
    best_models.append(best_model)

best_models = pd.concat(best_models)
best_models

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(metrics), 1, figsize=(20, 20))

# Add some space between the plots
fig.tight_layout(pad=10.0)

# Add some space between the plots
fig.tight_layout(pad=5.0)

fig.suptitle('Comparison of models')

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    scores = model_scores[model_scores['metric'] == metric]

    for _, data in scores.iterrows():
        ax.bar(data['model'], data['mean'], yerr=data['std'], capsize=5)

    ax.set_title(metric)
    ax.set_ylabel(metric)
    ax.set_xlabel('Model')

    # Rotate the x-axis labels
    for tick in ax.get_xticklabels():
        tick.set_rotation(0)

# Store the plot as svg
figure_dir = utils.dir_path('outputs', 'figures', mkdir=True)
utils.savefig(fig, figure_dir, 'model_comparison')